# Library Imports

In [3]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

# ANN Components Definition (Forward and Backward Pass)

In [4]:
# Dense Layer
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

# Activation Functions

In [5]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

# Loss Functions

In [6]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss


class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = y_pred.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        # Prevent divide by zero
        dvalues = np.clip(dvalues, 1e-7, 1 - 1e-7)
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / len(dvalues)

        self.dinputs = self.dinputs / samples

# Modified Optimizer Class that accepts 3 Optimizers Discussed

In [ ]:
class Optimizer_SGD:
    """
    Stochastic Gradient Descent with enhancements
    
    Parameters:
    - learning_rate: Initial learning rate
    - decay: Learning rate decay factor (0 = no decay)
    - momentum: Momentum factor, typically 0.9 (0 = no momentum)
    - adagrad: Enable AdaGrad adaptation (default: False)
    """
    
    def __init__(self, learning_rate=1.0, decay=0., momentum=0., adagrad=False):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        self.adagrad = adagrad
    
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate / (1. + self.decay * self.iterations)
    
    def update_params(self, layer):
        # === ADAGRAD IMPLEMENTATION ===
        if self.adagrad:
            # Initialize cache on first use
            if not hasattr(layer, 'weight_cache'):
                layer.weight_cache = np.zeros_like(layer.weights)
                layer.bias_cache = np.zeros_like(layer.biases)
            
            # Accumulate squared gradients
            layer.weight_cache += layer.dweights ** 2
            layer.bias_cache += layer.dbiases ** 2
            
            # Update with adapted learning rate
            layer.weights += -self.current_learning_rate * layer.dweights / \
                           (np.sqrt(layer.weight_cache) + 1e-7)
            layer.biases += -self.current_learning_rate * layer.dbiases / \
                          (np.sqrt(layer.bias_cache) + 1e-7)
        
        # === MOMENTUM IMPLEMENTATION ===
        elif self.momentum:
            # Initialize momentum arrays on first use
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            # Calculate momentum-based updates
            weight_updates = self.momentum * layer.weight_momentums - \
                           self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            
            bias_updates = self.momentum * layer.bias_momentums - \
                         self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
            
            # Apply updates
            layer.weights += weight_updates
            layer.biases += bias_updates
        
        # === VANILLA SGD ===
        else:
            layer.weights += -self.current_learning_rate * layer.dweights
            layer.biases += -self.current_learning_rate * layer.dbiases
    
    def post_update_params(self):
        self.iterations += 1

# Training Function

## - trains neural network for the specified number of epochs with performance tracking

In [8]:
def train_model(optimizer_name, learning_rate=1.0, decay=0., momentum=0., 
                adagrad=False, epochs=1000):
    
    print(f"\n{'='*70}")
    print(f"TRAINING WITH: {optimizer_name}")
    print(f"{'='*70}")
    
    # Create dataset
    nnfs.init()
    X, y = spiral_data(samples=100, classes=3)
    
    # Initialize network layers
    dense1 = Layer_Dense(2, 64)
    activation1 = Activation_ReLU()
    dense2 = Layer_Dense(64, 3)
    activation2 = Activation_Softmax()
    loss_function = Loss_CategoricalCrossEntropy()
    
    # Initialize optimizer
    optimizer = Optimizer_SGD(
        learning_rate=learning_rate,
        decay=decay,
        momentum=momentum,
        adagrad=adagrad
    )
    
    # Track metrics
    losses = []
    accuracies = []
    
    # === TRAINING LOOP ===
    for epoch in range(epochs):
        # FORWARD PASS
        dense1.forward(X)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        activation2.forward(dense2.output)
        
        # Calculate loss
        loss = loss_function.calculate(activation2.output, y)
        
        # Calculate accuracy
        predictions = np.argmax(activation2.output, axis=1)
        if len(y.shape) == 2:
            y_labels = np.argmax(y, axis=1)
        else:
            y_labels = y
        accuracy = np.mean(predictions == y_labels)
        
        # Store metrics
        losses.append(loss)
        accuracies.append(accuracy)
        
        # BACKWARD PASS
        loss_function.backward(activation2.output, y)
        activation2.backward(loss_function.dinputs)
        dense2.backward(activation2.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)
        
        # PARAMETER UPDATE
        optimizer.pre_update_params() # for application of learning rate decay
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)
        optimizer.post_update_params()
        
        # Display progress every 100 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch:4d} | Loss: {loss:.4f} | Accuracy: {accuracy:.4f} | LR: {optimizer.current_learning_rate:.6f}")
    
    # Final results
    print(f"\n{'─'*70}")
    print(f"FINAL RESULTS:")
    print(f"Final Loss: {losses[-1]:.4f}")
    print(f"Final Accuracy: {accuracies[-1]:.4f}")
    print(f"{'='*70}\n")
    
    return losses, accuracies

# VANILLA SGD WITH LEARNING RATE DECAY

In [9]:
losses_decay, acc_decay = train_model(
    optimizer_name="Vanilla SGD with Learning Rate Decay",
    learning_rate=0.5,
    decay=0.001,
    epochs=1000
)


TRAINING WITH: Vanilla SGD with Learning Rate Decay
Epoch    0 | Loss: 1.0986 | Accuracy: 0.3600 | LR: 0.500000
Epoch  100 | Loss: 1.0986 | Accuracy: 0.3633 | LR: 0.454545
Epoch  200 | Loss: 1.0986 | Accuracy: 0.3700 | LR: 0.416667
Epoch  300 | Loss: 1.0986 | Accuracy: 0.4067 | LR: 0.384615
Epoch  400 | Loss: 1.0986 | Accuracy: 0.4067 | LR: 0.357143
Epoch  500 | Loss: 1.0986 | Accuracy: 0.4067 | LR: 0.333333
Epoch  600 | Loss: 1.0986 | Accuracy: 0.3967 | LR: 0.312500
Epoch  700 | Loss: 1.0986 | Accuracy: 0.4033 | LR: 0.294118
Epoch  800 | Loss: 1.0986 | Accuracy: 0.4067 | LR: 0.277778
Epoch  900 | Loss: 1.0986 | Accuracy: 0.4033 | LR: 0.263158

──────────────────────────────────────────────────────────────────────
FINAL RESULTS:
Final Loss: 1.0986
Final Accuracy: 0.4033



# SGD WITH MOMENTUM

In [10]:
losses_momentum, acc_momentum = train_model(
    optimizer_name="SGD with Momentum",
    learning_rate=0.5,
    decay=0.001,
    momentum=0.9,
    epochs=1000
)


TRAINING WITH: SGD with Momentum
Epoch    0 | Loss: 1.0986 | Accuracy: 0.3600 | LR: 0.500000
Epoch  100 | Loss: 1.0986 | Accuracy: 0.3967 | LR: 0.454545
Epoch  200 | Loss: 1.0985 | Accuracy: 0.4000 | LR: 0.416667
Epoch  300 | Loss: 1.0985 | Accuracy: 0.4167 | LR: 0.384615
Epoch  400 | Loss: 1.0985 | Accuracy: 0.4167 | LR: 0.357143
Epoch  500 | Loss: 1.0985 | Accuracy: 0.4100 | LR: 0.333333
Epoch  600 | Loss: 1.0985 | Accuracy: 0.4100 | LR: 0.312500
Epoch  700 | Loss: 1.0985 | Accuracy: 0.4133 | LR: 0.294118
Epoch  800 | Loss: 1.0985 | Accuracy: 0.4133 | LR: 0.277778
Epoch  900 | Loss: 1.0984 | Accuracy: 0.4233 | LR: 0.263158

──────────────────────────────────────────────────────────────────────
FINAL RESULTS:
Final Loss: 1.0984
Final Accuracy: 0.4200



# ADAPTIVE GRADIENT (AdaGrad)

In [11]:
losses_adagrad, acc_adagrad = train_model(
    optimizer_name="AdaGrad (Adaptive Gradient)",
    learning_rate=1.0,
    decay=0.0,
    adagrad=True,
    epochs=1000
)


TRAINING WITH: AdaGrad (Adaptive Gradient)
Epoch    0 | Loss: 1.0986 | Accuracy: 0.3600 | LR: 1.000000
Epoch  100 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000
Epoch  200 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000
Epoch  300 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000
Epoch  400 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000
Epoch  500 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000
Epoch  600 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000
Epoch  700 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000
Epoch  800 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000
Epoch  900 | Loss: 10.7454 | Accuracy: 0.3333 | LR: 1.000000

──────────────────────────────────────────────────────────────────────
FINAL RESULTS:
Final Loss: 10.7454
Final Accuracy: 0.3333



# COMPARISON ANALYSIS

In [12]:
def find_stabilization_epoch(losses, threshold=0.01):
    for i in range(10, len(losses)):
        if abs(losses[i] - losses[i-10]) < threshold:
            return i
    return len(losses)

stabilize_decay = find_stabilization_epoch(losses_decay)
stabilize_momentum = find_stabilization_epoch(losses_momentum)
stabilize_adagrad = find_stabilization_epoch(losses_adagrad)

print("\n" + "="*70)
print("OPTIMIZER COMPARISON SUMMARY")
print("="*70)

print(f"\n1. Learning Rate Decay:")
print(f"   - Stabilization Epoch: ~{stabilize_decay}")
print(f"   - Final Accuracy: {acc_decay[-1]:.4f}")

print(f"\n2. Momentum:")
print(f"   - Stabilization Epoch: ~{stabilize_momentum}")
print(f"   - Final Accuracy: {acc_momentum[-1]:.4f}")

print(f"\n3. AdaGrad:")
print(f"   - Stabilization Epoch: ~{stabilize_adagrad}")
print(f"   - Final Accuracy: {acc_adagrad[-1]:.4f}")

print("\n" + "="*70)
print("KEY OBSERVATIONS:")
print("="*70)
print("• Momentum typically converges faster due to smoother updates")
print("• AdaGrad adapts learning rates per parameter")
print("• Learning rate decay prevents overshooting in later epochs")
print("="*70 + "\n")


OPTIMIZER COMPARISON SUMMARY

1. Learning Rate Decay:
   - Stabilization Epoch: ~10
   - Final Accuracy: 0.4033

2. Momentum:
   - Stabilization Epoch: ~10
   - Final Accuracy: 0.4200

3. AdaGrad:
   - Stabilization Epoch: ~13
   - Final Accuracy: 0.3333

KEY OBSERVATIONS:
• Momentum typically converges faster due to smoother updates
• AdaGrad adapts learning rates per parameter
• Learning rate decay prevents overshooting in later epochs

